# Basel III / FRTB VaR & ES framework for an electricity trading portfolio.

Features:
- 52 synthetic electricity instruments.
- Historical, Parametric, Monte Carlo VaR & ES.
- Portfolio mean and volatility.
- Volatility and correlation stress testing.
- Backtesting (Kupiec, Christoffersen).
- Plotly dashboard:
    Row 1: 3 VaR+ES histograms (Hist, Param, MC)
    Row 2: Exposure-weight bar chart (normalized weights)
    Row 3: Metrics table (VaR, ES, backtesting, vols, counts, etc.)
- Dashboard: Electricity_Risk_Report.html
- Excel log: risk_results.xlsx (including exposure weights).

# Part 1 - Imports, config, core classes

In [ ]:
import os
import yaml
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    
alpha = config["alpha"]                 # VaR / ES confidence level
n_days = config["n_days"]               # Number of historical days (synthetic)
n_mc = config["n_mc"]                   # Monte Carlo scenarios
kpi_filename = config["KPI_filename"]
dashboard_html = config["dashboard_name"]

In [ ]:
# ---------------------------------------------------------------------------
# Instrument and Portfolio Classes
# ---------------------------------------------------------------------------

class Instrument:
    """
    Represents a single electricity trading instrument.
    """

    def __init__(self, name: str, prices: pd.Series, exposure: float):
        self.name = name
        self.prices = prices
        self.exposure = exposure

    @property
    def returns(self) -> pd.Series:
        """
        Log-returns of the instrument:
        r_t = ln(P_t / P_{t-1})
        """
        return np.log(self.prices / self.prices.shift(1)).dropna()


class Portfolio:
    """
    Represents a portfolio of electricity instruments.
    """

    def __init__(self, instruments: list[Instrument]):
        self.instruments = instruments

    def get_returns_matrix(self) -> pd.DataFrame:
        """
        Returns a DataFrame of instrument returns aligned by date.
        """
        rets = {inst.name: inst.returns for inst in self.instruments}
        df = pd.DataFrame(rets).dropna()
        return df

    def get_exposure_vector(self) -> np.ndarray:
        """
        Returns exposure-weighted positions as a vector (w).
        """
        return np.array([inst.exposure for inst in self.instruments], dtype=float)

    def portfolio_return_series(self) -> pd.Series:
        """
        Portfolio return:
        R_{P,t} = sum_i w_i * r_{i,t}
        (weights normalized by absolute exposure sum)
        """
        returns_df = self.get_returns_matrix()
        w = self.get_exposure_vector()
        w_norm = w / np.sum(np.abs(w))
        rp = returns_df.dot(w_norm)
        rp.name = "Portfolio_Return"
        return rp

    def portfolio_loss_series(self) -> pd.Series:
        """
        Portfolio loss:
        L_t = - R_{P,t}
        """
        rp = self.portfolio_return_series()
        loss = -rp
        loss.name = "Portfolio_Loss"
        return loss


In [2]:
# ---------------------------------------------------------------------------
# VaR & ES Calculator
# ---------------------------------------------------------------------------

class VaRCalculator:
    """
    Computes Historical, Parametric, and Monte Carlo VaR & ES.
    """

    def __init__(self, portfolio: Portfolio, alpha: float = ALPHA):
        self.portfolio = portfolio
        self.alpha = alpha
        self.returns_df = portfolio.get_returns_matrix()
        self.loss_series = portfolio.portfolio_loss_series()

    # ---------------- Historical VaR & ES ----------------

    def historical_var(self) -> float:
        return np.quantile(self.loss_series.values, self.alpha)

    def historical_es(self) -> float:
        var = self.historical_var()
        tail_losses = self.loss_series[self.loss_series >= var]
        return tail_losses.mean()

    # ---------------- Parametric VaR & ES ----------------

    def parametric_inputs(self):
        rets = self.returns_df
        mu_vec = rets.mean().values
        sigma_mat = rets.cov().values
        w = self.portfolio.get_exposure_vector()
        w_norm = w / np.sum(np.abs(w))

        mu_p = w_norm @ mu_vec
        sigma_p2 = w_norm @ sigma_mat @ w_norm
        sigma_p = np.sqrt(sigma_p2)

        return mu_vec, sigma_mat, mu_p, sigma_p, w_norm

    def parametric_var(self) -> float:
        _, _, mu_p, sigma_p, _ = self.parametric_inputs()
        z = norm.ppf(self.alpha)
        var = -(mu_p + sigma_p * z)
        return var

    def parametric_es(self) -> float:
        _, _, mu_p, sigma_p, _ = self.parametric_inputs()
        z = norm.ppf(self.alpha)
        phi_z = norm.pdf(z)
        es = -(mu_p - sigma_p * phi_z / (1 - self.alpha))
        return es

    # ---------------- Monte Carlo VaR & ES ----------------

    def monte_carlo_var_es(self, n_scenarios: int = N_MC):
        rets = self.returns_df
        mu_vec = rets.mean().values
        sigma_mat = rets.cov().values
        w = self.portfolio.get_exposure_vector()
        w_norm = w / np.sum(np.abs(w))

        sim_rets = np.random.multivariate_normal(mu_vec, sigma_mat, size=n_scenarios)
        sim_port_rets = sim_rets @ w_norm
        sim_losses = -sim_port_rets

        var_mc = np.quantile(sim_losses, self.alpha)
        es_mc = sim_losses[sim_losses >= var_mc].mean()

        return var_mc, es_mc, sim_losses

# ---------------------------------------------------------------------------
# Backtesting
# ---------------------------------------------------------------------------

class Backtester:
    """
    Kupiec POF and Christoffersen independence tests.
    """

    def __init__(self, loss_series: pd.Series, var_series: pd.Series, alpha: float = ALPHA):
        self.loss_series = loss_series
        self.var_series = var_series
        self.alpha = alpha
        self.indicator = self._exceedance_indicator()

    def _exceedance_indicator(self) -> pd.Series:
        ind = (self.loss_series > self.var_series).astype(int)
        ind.name = "Exceedance_Indicator"
        return ind

    def kupiec_pof_test(self):
        I = self.indicator
        T = len(I)
        N = I.sum()
        p_hat = N / T
        p0 = 1 - self.alpha

        if p_hat in [0, 1]:
            LR_pof = 0.0
        else:
            L0 = (p0 ** N) * ((1 - p0) ** (T - N))
            L1 = (p_hat ** N) * ((1 - p_hat) ** (T - N))
            LR_pof = -2 * np.log(L0 / L1)

        p_value = 1 - chi2.cdf(LR_pof, df=1)
        return LR_pof, p_value, N, T

    def christoffersen_independence_test(self):
        I = self.indicator.values
        n00 = n01 = n10 = n11 = 0
        for t in range(1, len(I)):
            prev, curr = I[t-1], I[t]
            if prev == 0 and curr == 0:
                n00 += 1
            elif prev == 0 and curr == 1:
                n01 += 1
            elif prev == 1 and curr == 0:
                n10 += 1
            elif prev == 1 and curr == 1:
                n11 += 1

        n0 = n00 + n01
        n1 = n10 + n11

        pi01 = n01 / n0 if n0 > 0 else 0.0
        pi11 = n11 / n1 if n1 > 0 else 0.0

        N = I.sum()
        T = len(I)
        pi = N / T if T > 0 else 0.0

        def safe_pow(base, exp):
            return base ** exp if base > 0 else 1.0

        L0 = safe_pow(1 - pi, n00 + n10) * safe_pow(pi, n01 + n11)
        L1 = (safe_pow(1 - pi01, n00) * safe_pow(pi01, n01) *
              safe_pow(1 - pi11, n10) * safe_pow(pi11, n11))

        if L0 == 0 or L1 == 0:
            LR_ind = 0.0
        else:
            LR_ind = -2 * np.log(L0 / L1)

        p_value = 1 - chi2.cdf(LR_ind, df=1)
        return LR_ind, p_value, (n00, n01, n10, n11)

# ---------------------------------------------------------------------------
# Stress Testing
# ---------------------------------------------------------------------------

class StressTester:
    """
    Volatility and correlation stress testing.
    """

    def __init__(self, portfolio: Portfolio):
        self.portfolio = portfolio
        self.returns_df = portfolio.get_returns_matrix()
        self.w = portfolio.get_exposure_vector()
        self.w_norm = self.w / np.sum(np.abs(self.w))

    def base_covariance(self) -> np.ndarray:
        return self.returns_df.cov().values

    def base_portfolio_vol(self) -> float:
        sigma = self.base_covariance()
        sigma_p2 = self.w_norm @ sigma @ self.w_norm
        return np.sqrt(sigma_p2)

    def volatility_stress(self, k: float = 2.0):
        sigma = self.base_covariance()
        sigma_stress = (k ** 2) * sigma
        sigma_p_base = self.base_portfolio_vol()
        sigma_p_stress = k * sigma_p_base
        return sigma_stress, sigma_p_base, sigma_p_stress

    def correlation_stress(self, delta_rho: float = 0.2):
        sigma = self.base_covariance()
        stds = np.sqrt(np.diag(sigma))
        D = np.diag(stds)
        with np.errstate(divide='ignore', invalid='ignore'):
            R = sigma / np.outer(stds, stds)
            R = np.nan_to_num(R, nan=0.0)

        R_stress = np.minimum(R + delta_rho, 1.0)
        sigma_stress = D @ R_stress @ D

        sigma_p2_stress = self.w_norm @ sigma_stress @ self.w_norm
        sigma_p_stress = np.sqrt(sigma_p2_stress)
        sigma_p_base = self.base_portfolio_vol()

        return sigma_stress, sigma_p_base, sigma_p_stress

# ---------------------------------------------------------------------------
# Result Logging to Excel
# ---------------------------------------------------------------------------

def append_results_to_excel(results: dict, excel_path: str = RESULTS_EXCEL):
    """
    Append a single row of results to an Excel file.
    Index = timestamp of computation.
    """
    timestamp = pd.Timestamp(datetime.now())
    row = pd.DataFrame(results, index=[timestamp])

    if os.path.exists(excel_path):
        existing = pd.read_excel(excel_path, index_col=0)
        combined = pd.concat([existing, row])
    else:
        combined = row

    combined.to_excel(excel_path)


In [3]:
# ---------------------------------------------------------------------------
# Synthetic Data Generation and Instrument Creation
# ---------------------------------------------------------------------------

def generate_synthetic_prices(n_days: int, n_instruments: int, seed: int = 42) -> pd.DataFrame:
    np.random.seed(seed)
    dates = pd.date_range(end=datetime.today(), periods=n_days, freq="D")
    prices = {}
    for i in range(n_instruments):
        p0 = 50 + 10 * np.random.rand()
        mu = 0.0005
        sigma = 0.02
        rets = np.random.normal(mu, sigma, size=n_days)
        path = p0 * np.exp(np.cumsum(rets))
        prices[f"Instr_{i+1}"] = pd.Series(path, index=dates)
    return pd.DataFrame(prices)

def create_electricity_instruments(price_df: pd.DataFrame) -> list[Instrument]:
    instruments = []
    cols = price_df.columns

    names = []
    names += [f"MBL_{i+1}" for i in range(12)]
    names += [f"MPL_{i+1}" for i in range(12)]
    names += [f"Q_{i+1}" for i in range(4)]
    names += [f"M_{i+1}" for i in range(12)]
    names += [f"H_{i+1}" for i in range(12)]

    for name, col in zip(names, cols[:52]):
        prices = price_df[col]
        exposure = np.random.uniform(10, 100)
        instruments.append(Instrument(name=name, prices=prices, exposure=exposure))

    return instruments

# ---------------------------------------------------------------------------
# Plotly Dashboard
# ---------------------------------------------------------------------------

def build_dashboard(loss_series: pd.Series,
                    hist_var: float,
                    hist_es: float,
                    para_var: float,
                    para_es: float,
                    mc_losses: np.ndarray,
                    mc_var: float,
                    mc_es: float,
                    exposure_weights: dict,
                    metrics: dict,
                    html_path: str = DASHBOARD_HTML):
    """
    Dashboard layout:
    Row 1: 3 histograms (Hist, Param, MC) with VaR & ES lines.
    Row 2: Exposure weights bar chart (full width).
    Row 3: Metrics table (full width).
    """

    fig = make_subplots(
        rows=3, cols=3,
        specs=[
            [{"type": "histogram"}, {"type": "histogram"}, {"type": "histogram"}],
            [{"type": "bar", "colspan": 3}, None, None],
            [{"type": "table", "colspan": 3}, None, None],
        ],
        subplot_titles=("Historical VaR", "Parametric VaR", "Monte Carlo VaR",
                        "Exposure Weights", "Key Metrics")
    )

    # ---------------- Row 1: Historical VaR & ES ----------------
    fig.add_trace(
        go.Histogram(
            x=loss_series,
            nbinsx=50,
            name="Losses",
            marker_color="steelblue",
            showlegend=False
        ),
        row=1, col=1
    )
    fig.add_vline(x=hist_var, line=dict(color="red", dash="dash"), row=1, col=1)
    fig.add_annotation(
        x=hist_var, y=0.95, xref="x1", yref="paper",
        text=f"Hist VaR = {hist_var:.4f}",
        showarrow=True, arrowhead=2, ax=20, ay=-30,
        font=dict(color="red")
    )
    fig.add_vline(x=hist_es, line=dict(color="purple", dash="dot"), row=1, col=1)
    fig.add_annotation(
        x=hist_es, y=0.85, xref="x1", yref="paper",
        text=f"Hist ES = {hist_es:.4f}",
        showarrow=True, arrowhead=2, ax=20, ay=-30,
        font=dict(color="purple")
    )

    # ---------------- Row 1: Parametric VaR & ES ----------------
    fig.add_trace(
        go.Histogram(
            x=loss_series,
            nbinsx=50,
            name="Losses",
            marker_color="seagreen",
            showlegend=False
        ),
        row=1, col=2
    )
    fig.add_vline(x=para_var, line=dict(color="red", dash="dash"), row=1, col=2)
    fig.add_annotation(
        x=para_var, y=0.95, xref="x2", yref="paper",
        text=f"Param VaR = {para_var:.4f}",
        showarrow=True, arrowhead=2, ax=20, ay=-30,
        font=dict(color="red")
    )
    fig.add_vline(x=para_es, line=dict(color="purple", dash="dot"), row=1, col=2)
    fig.add_annotation(
        x=para_es, y=0.85, xref="x2", yref="paper",
        text=f"Param ES = {para_es:.4f}",
        showarrow=True, arrowhead=2, ax=20, ay=-30,
        font=dict(color="purple")
    )

    # ---------------- Row 1: Monte Carlo VaR & ES ----------------
    fig.add_trace(
        go.Histogram(
            x=mc_losses,
            nbinsx=50,
            name="Sim Losses",
            marker_color="darkorange",
            showlegend=False
        ),
        row=1, col=3
    )
    fig.add_vline(x=mc_var, line=dict(color="red", dash="dash"), row=1, col=3)
    fig.add_annotation(
        x=mc_var, y=0.95, xref="x3", yref="paper",
        text=f"MC VaR = {mc_var:.4f}",
        showarrow=True, arrowhead=2, ax=20, ay=-30,
        font=dict(color="red")
    )
    fig.add_vline(x=mc_es, line=dict(color="purple", dash="dot"), row=1, col=3)
    fig.add_annotation(
        x=mc_es, y=0.85, xref="x3", yref="paper",
        text=f"MC ES = {mc_es:.4f}",
        showarrow=True, arrowhead=2, ax=20, ay=-30,
        font=dict(color="purple")
    )

    # ---------------- Row 2: Enhanced Exposure Weights Plot ----------------

    # 1. Sort exposure weights ascending
    sorted_items = sorted(exposure_weights.items(), key=lambda x: x[1])
    inst_names = [item[0] for item in sorted_items]
    inst_weights = [item[1] for item in sorted_items]

    # 2. Category color-coding
    def category_color(name):
        if name.startswith("MBL"): return "royalblue"
        if name.startswith("MPL"): return "seagreen"
        if name.startswith("Q_"):  return "darkorange"
        if name.startswith("M_"):  return "purple"
        if name.startswith("H_"):  return "brown"
        return "grey"

    base_colors = [category_color(n) for n in inst_names]

    # 3. Highlight top 5 exposures
    top5_threshold = sorted(inst_weights)[-5]
    colors = [
        "crimson" if w >= top5_threshold else base_colors[i]
        for i, w in enumerate(inst_weights)
    ]

    # 4. Add bar chart (exposure weights)
    fig.add_trace(
        go.Bar(
            x=inst_names,
            y=inst_weights,
            marker_color=colors,
            hovertemplate="<b>%{x}</b><br>Weight: %{y:.6f}<extra></extra>",
            name="Exposure Weights"
        ),
        row=2, col=1
    )

    # 5. Add cumulative exposure curve (Pareto)
    cum_weights = np.cumsum(inst_weights)

    fig.add_trace(
        go.Scatter(
            x=inst_names,
            y=cum_weights,
            mode="lines+markers",
            name="Cumulative Weight",
            yaxis="y2",
            line=dict(color="black", width=2)
        ),
        row=2, col=1
    )

    # 6. Secondary y-axis for cumulative curve
    fig.update_layout(
        yaxis2=dict(
            overlaying="y",
            side="right",
            title="Cumulative Weight"
        )
    )

    # 7. Axis formatting
    fig.update_xaxes(
        tickangle=45,
        row=2, col=1
    )

    fig.update_yaxes(
        title_text="Weight",
        row=2, col=1
    )


    # ---------------- Row 3: Metrics Table ----------------
    table_header = ["Metric", "Value"]
    table_values = [
        list(metrics.keys()),
        [f"{v:.6f}" if isinstance(v, (int, float)) else str(v) for v in metrics.values()]
    ]

    fig.add_trace(
        go.Table(
            header=dict(values=table_header, fill_color="lightgrey", align="left"),
            cells=dict(values=table_values, align="left")
        ),
        row=3, col=1
    )

    fig.update_layout(
        title="Electricity Portfolio VaR & ES Dashboard",
        bargap=0.1,
        template="plotly_white",
        height=1000,

        # --- Vertical hover line enhancement ---
        hovermode="x",
        spikedistance=-1,

        xaxis=dict(showspikes=True, spikemode="across", spikesnap="cursor", spikethickness=1),
        xaxis2=dict(showspikes=True, spikemode="across", spikesnap="cursor", spikethickness=1),
        xaxis3=dict(showspikes=True, spikemode="across", spikesnap="cursor", spikethickness=1),
        xaxis4=dict(showspikes=True, spikemode="across", spikesnap="cursor", spikethickness=1),
    )

    fig.update_xaxes(title_text="Loss", row=1, col=1)
    fig.update_xaxes(title_text="Loss", row=1, col=2)
    fig.update_xaxes(title_text="Loss", row=1, col=3)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)

    fig.write_html(html_path)


In [4]:
# ---------------------------------------------------------------------------
# Main Execution
# ---------------------------------------------------------------------------

def main():
    # 1. Synthetic prices and instruments
    n_instruments = 52
    price_df = generate_synthetic_prices(N_DAYS, n_instruments)
    instruments = create_electricity_instruments(price_df)

    # 2. Portfolio
    portfolio = Portfolio(instruments=instruments)

    # 3. Portfolio returns and losses
    port_returns = portfolio.portfolio_return_series()
    port_losses = portfolio.portfolio_loss_series()

    # 4. VaR & ES
    var_calc = VaRCalculator(portfolio=portfolio, alpha=ALPHA)

    hist_var = var_calc.historical_var()
    hist_es = var_calc.historical_es()

    mu_vec, sigma_mat, mu_p, sigma_p, w_norm = var_calc.parametric_inputs()
    para_var = var_calc.parametric_var()
    para_es = var_calc.parametric_es()

    mc_var, mc_es, mc_losses = var_calc.monte_carlo_var_es(N_MC)

    # 5. Backtesting (Historical VaR)
    var_series_hist = pd.Series(hist_var, index=port_losses.index, name="VaR_Hist")
    backtester_hist = Backtester(loss_series=port_losses, var_series=var_series_hist, alpha=ALPHA)
    kupiec_LR, kupiec_p, N_exc, T_obs = backtester_hist.kupiec_pof_test()
    christ_LR, christ_p, trans_counts = backtester_hist.christoffersen_independence_test()

    # 6. Stress testing
    stress_tester = StressTester(portfolio=portfolio)
    sigma_stress_vol, sigma_p_base_vol, sigma_p_stress_vol = stress_tester.volatility_stress(k=2.0)
    sigma_stress_corr, sigma_p_base_corr, sigma_p_stress_corr = stress_tester.correlation_stress(delta_rho=0.2)

    # 7. Normalized exposure weights (for Excel + dashboard)
    exposures = portfolio.get_exposure_vector()
    w_norm = exposures / np.sum(np.abs(exposures))
    exposure_weights = {inst.name: float(w) for inst, w in zip(instruments, w_norm)}

    # 8. Collect metrics for Excel and table
    results = {
        "Portfolio_Mean": mu_p,
        "Portfolio_Volatility": sigma_p,
        "Hist_VaR": hist_var,
        "Hist_ES": hist_es,
        "Para_VaR": para_var,
        "Para_ES": para_es,
        "MC_VaR": mc_var,
        "MC_ES": mc_es,
        "Kupiec_LR": kupiec_LR,
        "Kupiec_p_value": kupiec_p,
        "Christoffersen_LR": christ_LR,
        "Christoffersen_p_value": christ_p,
        "N_exceedances": N_exc,
        "T_observations": T_obs,
        "Base_Portfolio_Volatility": sigma_p_base_vol,
        "Vol_Stress_Portfolio_Volatility": sigma_p_stress_vol,
        "Corr_Stress_Base_Volatility": sigma_p_base_corr,
        "Corr_Stress_Portfolio_Volatility": sigma_p_stress_corr,
    }

    # Add exposure weights to Excel metrics
    for name, w in exposure_weights.items():
        results[f"{name}_weight"] = w

    append_results_to_excel(results, RESULTS_EXCEL)

    # 9. Build dashboard
    build_dashboard(
        loss_series=port_losses,
        hist_var=hist_var,
        hist_es=hist_es,
        para_var=para_var,
        para_es=para_es,
        mc_losses=mc_losses,
        mc_var=mc_var,
        mc_es=mc_es,
        exposure_weights=exposure_weights,
        metrics=results,
        html_path=DASHBOARD_HTML
    )

    print("Computation complete.")
    print(f"Dashboard saved to: {DASHBOARD_HTML}")
    print(f"Results appended to: {RESULTS_EXCEL}")

if __name__ == "__main__":
    main()


Computation complete.
Dashboard saved to: Electricity_Risk_Report.html
Results appended to: risk_results.xlsx
